In [16]:
import yake
from summarizer import Summarizer
from keybert import KeyBERT
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [17]:
def bert_summ(sample_text):
    
#     model = Summarizer()
#     result = model(text, ratio=0.4)
#     summarized_text = ''.join(result)
#     return summarized_text

    tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
    model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")


    # Tokenize and summarize the text
    inputs = tokenizer.encode("summarize: " + sample_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [12]:
def keyword_selection(text):
    kw_extractor = yake.KeywordExtractor()
    keywords_yake = kw_extractor.extract_keywords(text)
    kw_model = KeyBERT()
    keywords_bert = kw_model.extract_keywords(text)
    keywords_yake_l = [i[0] for i in keywords_yake]
    keywords_bert_l = [i[0] for i in keywords_bert]
    keywords = keywords_yake_l + keywords_bert_l
    return keywords

In [13]:
class SentenceMapping:
    
    def tokenize_sentences(text):
        sentences = [sent_tokenize(text)]
        sentences = [y for x in sentences for y in x]
        sentences = [sentence.strip() for sentence in sentences if len(sentence)>20]
        return sentences
    
    def get_sentences_for_keyword(keywords, sentences):
        keyword_processor = KeywordProcessor()
        keyword_sentences = {}
        for word in keywords:
            keyword_sentences[word] = []
            keyword_processor.add_keyword(word)
        for sentence in sentences:
            keywords_found = keyword_processor.extract_keywords(sentence)
            for key in keywords_found:
                keyword_sentences[key].append(sentence)
        for key in keyword_sentences.keys():
            values = keyword_sentences[key]
            values = sorted(values, key=len, reverse=True)
            keyword_sentences[key] = values
        lis = []
        for x, y in keyword_sentences.items():
            if len(y) == 0:
                lis.append(x)
            elif len(y) > 5:
                lis.append(x)
        for i in lis:
            del keyword_sentences[i]
        return keyword_sentences